In [55]:
import os
import boto3
from config import *
import base64
import json

In [56]:
print(boto3.__version__)

1.37.4


In [57]:
bedrock_agent = boto3.Session().client('bedrock-agent')
bedrock_rt = boto3.Session().client('bedrock-runtime')

In [58]:
image_path = "resources/cat_image.png"

In [59]:
def image_to_bytes(image_path):
    with open(image_path, "rb") as image_file:
        # Read the file content
        binary_data = image_file.read()
        # Encode the binary data to base64
        base64_encoded = base64.b64encode(binary_data)
        # Convert bytes to string
        base64_string = base64_encoded.decode('utf-8')      # <---- adding this step
    return {'data': base64_string}

In [60]:
image_bytes = image_to_bytes(image_path)
image_bytes_data = image_bytes['data']
images = [image_bytes_data]

In [61]:
def invoke_llama(modelId):
    # Embed the prompt in Llama 3's instruction format.
    formatted_prompt = f"""
    <|begin_of_text|><|start_header_id|>user<|end_header_id|><|image|>Describe this image in two
    sentences
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """
    
    payload = {
        "prompt": formatted_prompt,
        "images": images,
        "max_gen_len": 2048,
        "temperature": 0.0
    }
    
    # Convert the payload to JSON
    request_body = json.dumps(payload)
    
    # Invoke the model
    response = bedrock_rt.invoke_model(
        modelId=model_id,
        body=request_body,
        contentType='application/json'
    )

    # Assuming your response is stored in a variable called 'response'
    body = response['body']
    content = body.read().decode('utf-8')
    
    # Parse the JSON content
    parsed_content = json.loads(content)
    
    print(json.dumps(parsed_content, indent=2))

In [62]:
models_and_inference_profiles = ["us.meta.llama3-2-90b-instruct-v1:0", "meta.llama3-2-11b-instruct-v1:0", "us.meta.llama3-2-11b-instruct-v1:0"]

In [63]:
for inference_id in models_and_inference_profiles:
    invoke_llama(inference_id)
    

{
  "generation": " The image depicts a cat lying on the ground, with its head up and looking directly at the camera. The cat has a striped coat with white paws and a white chest, and it appears to be in a relaxed pose.",
  "prompt_token_count": 25,
  "generation_token_count": 47,
  "stop_reason": "stop"
}
{
  "generation": " The image depicts a cat lying on the ground, with its head up and looking directly at the camera. The cat has a striped coat with white paws and a white chest, and it appears to be in a relaxed pose.",
  "prompt_token_count": 25,
  "generation_token_count": 47,
  "stop_reason": "stop"
}
{
  "generation": " The image depicts a cat lying on the ground, with its head up and looking directly at the camera. The cat has a striped coat with white paws and a white chest, and it appears to be in a relaxed pose.",
  "prompt_token_count": 25,
  "generation_token_count": 47,
  "stop_reason": "stop"
}
